# Tweeteval sentiment detection

In [12]:
import pandas as pd
from pathlib import Path
import swifter
from scipy.stats import entropy
from collections import Counter

In [3]:
!git clone https://github.com/cardiffnlp/tweeteval.git

/bin/bash: /home/oliwier/anaconda3/envs/representation/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Cloning into 'tweeteval'...
remote: Enumerating objects: 370, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 370 (delta 13), reused 1 (delta 1), pack-reused 354
Receiving objects: 100% (370/370), 8.49 MiB | 18.50 MiB/s, done.
Resolving deltas: 100% (122/122), done.


In [4]:
DATA_DIR = Path("tweeteval") / "datasets"
SENTIMENT_DIR = DATA_DIR / "sentiment"
GPT_INPUT_PATH = Path("gpt-input")

GPT_INPUT_PATH.mkdir(exist_ok=True)

In [5]:
MAPPING = {
    "0":	"negative",
    "1":	"neutral",
    "2":	"positive"

}

RESULTS = ["0", "1", "2"]

In [6]:
MAIN_STORY = """
Describe the sentiment of the given text. Choose your answer from provided list and
map your answer with following negative: 0, neutral: 1, positive: 2 and return an integer as a result. Do not explain yourself.
Input data:

Text: {}

Possible sentiment: negative, neutral, positive
"""

In [7]:
def load_test_data(dataset_path: Path):
    df_text = pd.read_csv(dataset_path / "test_text.txt", sep="delimiter", header=None, names=["text"])
    df_labels = pd.read_csv(dataset_path / "test_labels.txt", sep="delimiter", header=None, names=["labels"])
    return pd.concat([df_text, df_labels], axis=1)

In [8]:
df = load_test_data(SENTIMENT_DIR)
df

/tmp/ipykernel_13229/578883216.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_text = pd.read_csv(dataset_path / "test_text.txt", sep="delimiter", header=None, names=["text"])
/tmp/ipykernel_13229/578883216.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_labels = pd.read_csv(dataset_path / "test_labels.txt", sep="delimiter", header=None, names=["labels"])


,text,labels
0,@user @user what do these '1/2 naked pics' hav...,1
1,OH: “I had a blue penis while I was this” [pla...,1
2,"@user @user That's coming, but I think the vic...",1
3,I think I may be finally in with the in crowd ...,2
4,"@user Wow,first Hugo Chavez and now Fidel Cast...",0
...,...,...
12279,Sentinel Editorial: FBI’s Comey ‘had no one of...,1
12280,perfect pussy clips #vanessa hudgens zac efron...,1
12281,#latestnews 4 #newmexico #politics + #nativeam...,1
12282,Trying to have a conversation with my dad abou...,0


In [14]:
entropy(list(Counter(df.labels).values())) # dataset entropy

1.0341982678002202

In [9]:
def generate_dataframe(df: pd.DataFrame):
    df["prompt"] = df["text"].swifter.apply(lambda row: MAIN_STORY.format(row))
    df = df.drop(["text", "labels"], axis = 1)
    df = df.reset_index(names="id")
    return df

In [13]:
new_df = generate_dataframe(load_test_data(SENTIMENT_DIR))
new_df.to_csv(GPT_INPUT_PATH / "sentiment.csv", index=False)

/tmp/ipykernel_12016/578883216.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_text = pd.read_csv(dataset_path / "test_text.txt", sep="delimiter", header=None, names=["text"])
/tmp/ipykernel_12016/578883216.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_labels = pd.read_csv(dataset_path / "test_labels.txt", sep="delimiter", header=None, names=["labels"])


Pandas Apply:   0%|          | 0/12284 [00:00<?, ?it/s]